In [21]:
import sys
sys.path.append('..')


In [22]:
from sklearn.model_selection import train_test_split
import pandas as pd
from torch import nn
from transformers import AutoTokenizer, AutoModel
import torch
!set 'PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:512'


In [23]:
df = pd.read_csv("../data/train.csv")
df_test = pd.read_csv("../data/test.csv")


In [24]:
labels = 'toxic severe_toxic obscene threat insult identity_hate'.split()
input_dim = 768
learning_rate = 0.003
batch_size = 16
epoch_num = 1
weight_decay = 0.01


In [25]:
df.head()


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\r\nWhy the edits made under my use...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\r\nMore\r\nI can't make any real suggestions...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [26]:
device = torch.device('cuda')
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModel.from_pretrained("distilbert-base-uncased").to(device)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [27]:
from src.utils.preprocessing import Processor
processor = Processor(labels, tokenizer, device)
train, val = train_test_split(df, test_size=0.3, train_size=0.7, shuffle=True)


In [28]:
X_train = processor.tokenize_data(train)
X_val = processor.tokenize_data(val)
y_train = processor.extract_target(train)
y_val = processor.extract_target(val)


In [29]:
from torch.utils.data import DataLoader, TensorDataset
from src.models.classifier import Classifier
dataset = TensorDataset(X_train, y_train)
dataloader = DataLoader(dataset, batch_size=batch_size)
test_dataloader = DataLoader(TensorDataset(
    X_val, y_val), batch_size=batch_size)

classifier = Classifier(input_dim, device).to(device)
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(
    classifier.parameters(), lr=learning_rate, weight_decay=weight_decay)


In [30]:
from src.models.train import train_loop, test_loop
for epoch in range(epoch_num):
    train_loop(dataloader, model, classifier, loss_fn, optimizer)
    test_loop(
        test_dataloader, model, classifier, loss_fn)


1it [00:00,  1.36it/s]

loss: 0.692744  [   16/ 3500]


219it [02:02,  1.79it/s]
100%|██████████| 94/94 [00:19<00:00,  4.77it/s]

Test Error: 
 Accuracy: [0.90466667 0.99266667 0.948      0.996      0.95       0.99066667], Avg loss: 0.141496 

